In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
df=pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
df=df.drop(columns=["id"])

In [3]:
df.isnull().sum()

MonsoonIntensity                   0
TopographyDrainage                 0
RiverManagement                    0
Deforestation                      0
Urbanization                       0
ClimateChange                      0
DamsQuality                        0
Siltation                          0
AgriculturalPractices              0
Encroachments                      0
IneffectiveDisasterPreparedness    0
DrainageSystems                    0
CoastalVulnerability               0
Landslides                         0
Watersheds                         0
DeterioratingInfrastructure        0
PopulationScore                    0
WetlandLoss                        0
InadequatePlanning                 0
PoliticalFactors                   0
FloodProbability                   0
dtype: int64

In [4]:
X=df.drop(columns=["FloodProbability"])
Y=df["FloodProbability"]

In [5]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [6]:
# random_reg=RandomForestRegressor()
# random_reg.fit(X_train,Y_train)
# random_reg.score(X_test,Y_test)

In [7]:
X_train.shape

(894365, 20)

In [8]:

Y_train

805665     0.575
863045     0.400
496045     0.505
1079364    0.455
191800     0.535
           ...  
110268     0.450
259178     0.545
131932     0.420
671155     0.485
121958     0.505
Name: FloodProbability, Length: 894365, dtype: float64

# # Deep learning

In [14]:
from keras.layers import Dense,Dropout,BatchNormalization,Input
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [15]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [24]:
inputs=Input(shape=(20,))
d=Dense(32,activation="relu")(inputs)
d=Dense(64,activation="relu")(d)
outputs=Dense(1)(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])

In [28]:
for layer in model.layers:
    layer.trainable=False
model_data=model.layers[-1].output

In [25]:
model.fit(X_train,Y_train,epochs=50,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - loss: 0.0090 - mean_absolute_error: 0.0377 - val_loss: 4.6446e-04 - val_mean_absolute_error: 0.0161
Epoch 2/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 4.8939e-04 - mean_absolute_error: 0.0174 - val_loss: 4.1487e-04 - val_mean_absolute_error: 0.0164
Epoch 3/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 4.5462e-04 - mean_absolute_error: 0.0167 - val_loss: 4.0723e-04 - val_mean_absolute_error: 0.0158
Epoch 4/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 4.3584e-04 - mean_absolute_error: 0.0164 - val_loss: 4.0654e-04 - val_mean_absolute_error: 0.0160
Epoch 5/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step - loss: 4.3235e-04 - mean_absolute_error: 0.0163 - val_loss: 4.1637e-04 - val_mean_absolute_error: 0.0165
Epoch 6/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step - loss: 4.2425e-04 - mean_absolute_error: 0.0162 - val_loss: 4.2205e-04 - val_mean_absolute_error: 0.0156
Epoch 7/50
22360/22360 ━━━

In [33]:
loss,mae=model.evaluate(X_test,Y_test)

6988/6988 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 13.4785 - mean_absolute_error: 0.8713


# # M2

In [30]:

d=Dense(526,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(model_data)
d=Dropout(0.2)(d)
d=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(d)
b=BatchNormalization()(d)
d=Dense(5,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)

outputs2=Dense(1)(d)
model2=Model(inputs,outputs2)
model2.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])

In [31]:
model2.fit(X_train,Y_train,epochs=50,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - loss: 1.7095 - mean_absolute_error: 0.0837 - val_loss: 9.8419e-04 - val_mean_absolute_error: 0.0204
Epoch 2/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 0.0010 - mean_absolute_error: 0.0206 - val_loss: 7.7306e-04 - val_mean_absolute_error: 0.0179
Epoch 3/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 9.6721e-04 - mean_absolute_error: 0.0211 - val_loss: 0.0013 - val_mean_absolute_error: 0.0279
Epoch 4/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 9.7122e-04 - mean_absolute_error: 0.0215 - val_loss: 0.0012 - val_mean_absolute_error: 0.0265
Epoch 5/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 9.9622e-04 - mean_absolute_error: 0.0219 - val_loss: 7.0166e-04 - val_mean_absolute_error: 0.0192
Epoch 5: early stopping


In [34]:
for layer in model2.layers:
    layer.trainable=False
model2_data=model.layers[-1].output

In [32]:
loss,mae=model2.evaluate(X_test,Y_test)

6988/6988 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 6.9878e-04 - mean_absolute_error: 0.0191


# # M3

In [36]:

d=Dense(1024,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(model2_data)
d=Dropout(0.2)(d)
d=Dense(526,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(d)
b=BatchNormalization()(d)
d=Dense(1150,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(b)

outputs3=Dense(1)(d)
model3=Model(inputs,outputs2)
model3.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])

In [37]:
model3.fit(X_train,Y_train,epochs=50,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


22360/22360 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - loss: 7.3779e-04 - mean_absolute_error: 0.0205 - val_loss: 5.4633e-04 - val_mean_absolute_error: 0.0192
Epoch 2/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 7.3875e-04 - mean_absolute_error: 0.0205 - val_loss: 5.4633e-04 - val_mean_absolute_error: 0.0192
Epoch 3/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 7.3629e-04 - mean_absolute_error: 0.0205 - val_loss: 5.4633e-04 - val_mean_absolute_error: 0.0192
Epoch 4/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 7.3957e-04 - mean_absolute_error: 0.0205 - val_loss: 5.4633e-04 - val_mean_absolute_error: 0.0192
Epoch 5/50
22360/22360 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 7.3848e-04 - mean_absolute_error: 0.0206 - val_loss: 5.4633e-04 - val_mean_absolute_error: 0.0192
Epoch 5: early stopping


In [38]:
loss,mae=model3.evaluate(X_test,Y_test)

6988/6988 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - loss: 5.4344e-04 - mean_absolute_error: 0.0191
